# Visualising Delaware School Locations

## 1. Installing dependencies

Firstly we have to make sure all the python packages we wish to use are installed.

Prefixing a line with `!` will run the rest of the line as a shell command.

The following is just like using `pip` from the shell.

In [3]:
! pip install folium==0.3.0 pandas requests

## 2. Importing packages before we start working

Before we can use them we have to import the dependecies we installed in step 1.

In [21]:
# Import builtin python packages
import math
import os

# Import installed python packages
import requests
import pandas as pd
print(pd.__version__)
import folium

0.19.2


## 3. Downloading data

### GeoJSON shapes for Delaware

https://en.wikipedia.org/wiki/GeoJSON
> GeoJSON[1] is an open standard format designed for representing simple geographical features ...

State, county, census tract, congressional district, and zip code geographic geojson shapes can be found at https://github.com/tomdottom/delaware_geojson 


In [22]:
# Download Delaware zip code geojson shapes
    
restaurant_inspection_violations_url = 'https://raw.githubusercontent.com/tomdottom/delaware_geojson/master/zcta.geojson'
# Note: The notebooks.azure.com service limits where data can be downloaded from.
#       We've downloaded the data to github with IS accessible within notebooks.azure.com
#       Just uncomment the url below and comment out the one above
# restaurant_inspection_violations_url = 'https://raw.githubusercontent.com/tomdottom/putting-data-on-the-web/master/data/de_zcta.geojson'

# Download the data
with open('./data/de_zcta.geojson', 'wb') as fh:
    response = requests.get(restaurant_inspection_violations_url)
    if response.status_code == 200:
        fh.write(response.content)
        print("Download succeeded")
    else:
        print("Download failed")
        print(response.reason)

Download succeeded


### Delaware School Directory Data

Next lets download some data from _http://data.delaware.gov_.

The Education category contains a dataset called _School Directory_

In [23]:
# Download Delaware School Directory json data
    
school_directory_url = 'http://data.delaware.gov/resource/f6a3-crpj.json'
# Note: The notebooks.azure.com service limits where data can be downloaded from.
#       We've downloaded the data to github with IS accessible within notebooks.azure.com
#       Just uncomment the url below and comment out the one above
# school_directory_url = 'https://raw.githubusercontent.com/WilmingtonWebDev/delaware_geojson/master/delaware_school_directory.json'

# Download the data
with open('./data/delaware_school_directory.json', 'wb') as fh:
    response = requests.get(school_directory_url)
    if response.status_code == 200:
        fh.write(response.content)
        print("Download succeeded")
    else:
        print("Download failed")
        print(response.reason)

Download succeeded


In [24]:
df = pd.read_json('./data/delaware_school_directory.json')

In [72]:
df.head(3)

,city,county,districtcode,districtname,districttype,educationlevel,geocoded_location,geocoded_location_address,geocoded_location_city,geocoded_location_state,...,isungraded,lowestgrade,schoolcode,schoolname,schooltype,schoolyear,state,street1,street2,zip
0,Magnolia,Kent,10,Caesar Rodney School District,Regular,Kindergarten,NaN,Post Office Box 258,Magnolia,DE,...,0,KG,611,J. Ralph McIlvaine Early Childhood Center,PUBLIC,2016,DE,Post Office Box 258,11 East Walnut,19962
1,Dover,Kent,10,Caesar Rodney School District,Regular,Elementary,NaN,3100 Hawthorne Drive,Dover,DE,...,0,01,612,Major George S. Welch Elementary School,PUBLIC,2016,DE,3100 Hawthorne Drive,--,199015391
2,Dover,Kent,10,Caesar Rodney School District,Regular,Middle,NaN,3100 Hawthorne Drive,Dover,DE,...,0,06,628,Dover Air Force Base Middle School,PUBLIC,2016,DE,3100 Hawthorne Drive,--,199015391


In [70]:
# Create a map centered and zoomed into Delaware
school_map = folium.Map(location=[39, -75], zoom_start=8)

# note you can increase this, but it may make you notebook crash
schools_to_show = 100
print("Showing {} out of {} schools".format(schools_to_show, len(df)))

for index, row in df[:schools_to_show].iterrows():
    location = row['geocoded_location']
    school_name = row['schoolname']
    if isinstance(location, dict):
        fixedLoc = [location['coordinates'][1], location['coordinates'][0]]
        folium.Marker(fixedLoc, popup=school_name).add_to(school_map)

Showing 100 out of 1000 schools


In [71]:
school_map

[fin]